# Plot

In [10]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import sys
import random
sys.path.append('../')
from auxPlots import getInfo, getDistributions, getATLASdistributions,getLHEevents

pd.option_context('display.max_columns', -1)

pd.options.mode.chained_assignment = None #Disable copy warnings
# plt.style.use('fivethirtyeight') #Set style
# mpl.rcParams.update({'figure.figsize' : (15,10)})  #Set general plotting options
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})

plt.rcParams.update({"savefig.dpi" : 300}) #Figure resolution


#Define plotting style:
sns.set_style('ticks',{'font.family':'Times New Roman', 'font.serif':'Times New Roman'})
sns.set_context('paper', font_scale=1.8)
cm = plt.colormaps['RdYlBu']

In [16]:
files = list(glob.glob('../../processFolders/pp_formFactors_scan/Events/*decayed*/unweighted_events.lhe.gz'))

In [17]:
# Get info
fileData = {f : getInfo(f) for f in files}
# Set SM model by hand:

allModels = sorted(list(set([info['model'] for info in fileData.values()])))
allPars = sorted(list(set([info['(mST,mChi,mT,yDM)'] for info in fileData.values()])))
allProcesses = sorted(list(set([info['process'] for info in fileData.values()])))
print(allModels)
print(allPars)
print(allProcesses)

['1-loop']
[(500.0, 400.0, 172.5, 1.0)]
['$p p \\to \\bar{t} t$']


In [18]:
# Select files:
models = ['1-loop']
pars = [(500.,400.,172.5,1.0)]
procs = allProcesses[:]

bsmFiles = []
smFiles = []
for f,fInfo in fileData.items():
    if fInfo['model'] not in models:
        continue
    if fInfo['process'] not in procs:
        continue
        
    if fInfo['model'] == 'SM':
        smFiles.append(f)
    else:
        if fInfo['(mST,mChi,mT,yDM)'] not in pars:
            continue
        bsmFiles.append(f)
    
# print(bsmFiles)
# print(smFiles)
# Sort files so largest cross-sections come first
bsmFiles = sorted(bsmFiles, key = lambda f: (fileData[f]['process'],fileData[f]['model']), reverse=True)
smFiles = sorted(smFiles, key = lambda f: fileData[f]['process'], reverse=True)
selectedFiles = smFiles+bsmFiles

In [19]:
print(len(selectedFiles))
for f in selectedFiles:
    print(f)

1
../../processFolders/pp_formFactors_scan/Events/run_01_decayed_1/unweighted_events.lhe.gz


In [20]:
for f in selectedFiles:
    nevents,events = getLHEevents(f)
    break

In [21]:
for ev in events:
    for ptc in ev.particles:
        print(ptc.id,ptc.status)
    break
    

1.0 -1.0
-1.0 -1.0
6.0 2.0
5.0 1.0
24.0 2.0
16.0 1.0
-15.0 1.0
-6.0 2.0
-5.0 1.0
-24.0 2.0
1.0 1.0
-2.0 1.0


### Get final states without cuts

In [23]:
dists = {f : {} for f in selectedFiles}
for f in selectedFiles:
    nevents,events = getLHEevents(f)
    for ev in events:
        finalStates = tuple(sorted([abs(ptc.id) for ptc in ev.particles if ptc.status == 1.0]))
        if finalStates not in dists[f]:
            dists[f][finalStates] = 0
        dists[f][finalStates] += 1

In [27]:
for f in dists:
    for fs in sorted(dists[f].keys()):
        print(fs,float(dists[f][fs])/nevents)

(1.0, 1.0, 2.0, 2.0, 5.0, 5.0) 0.12
(1.0, 2.0, 3.0, 4.0, 5.0, 5.0) 0.224
(1.0, 2.0, 5.0, 5.0, 11.0, 12.0) 0.072
(1.0, 2.0, 5.0, 5.0, 13.0, 14.0) 0.068
(1.0, 2.0, 5.0, 5.0, 15.0, 16.0) 0.072
(3.0, 3.0, 4.0, 4.0, 5.0, 5.0) 0.104
(3.0, 4.0, 5.0, 5.0, 11.0, 12.0) 0.078
(3.0, 4.0, 5.0, 5.0, 13.0, 14.0) 0.075
(3.0, 4.0, 5.0, 5.0, 15.0, 16.0) 0.081
(5.0, 5.0, 11.0, 11.0, 12.0, 12.0) 0.006
(5.0, 5.0, 11.0, 12.0, 13.0, 14.0) 0.033
(5.0, 5.0, 11.0, 12.0, 15.0, 16.0) 0.02
(5.0, 5.0, 13.0, 13.0, 14.0, 14.0) 0.01
(5.0, 5.0, 13.0, 14.0, 15.0, 16.0) 0.019
(5.0, 5.0, 15.0, 15.0, 16.0, 16.0) 0.018


In [28]:
nhadronic = 0.12+0.224+0.104 # expected = 0.6741*0.6741 = 0.4544
nsemileptonic = 0.072+0.068+0.072+0.078+0.075+0.081 # expected = 2*0.6741*(1-0.6741) = 0.439378
nleptonic = 0.006+0.033+0.02+0.01+0.019+0.018 # expected = (1-0.6741)*(1-0.6741) = 0.1062
print(nhadronic,nsemileptonic,nleptonic)

0.44799999999999995 0.44600000000000006 0.106
